In [66]:
import networkx as nx
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import ttk
import csv

# Columnas
player = []
nation = []
position = []
squad = []
competition = []
age = []
born = []
time = []
goal = []
assist = []
dribbling = []
tackle = []

# Abre el archivo CSV
with open('Base.csv', 'r', newline='') as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=';')
    next(csv_reader)
    for row in csv_reader:
        player.append(row[0])
        nation.append(row[1])
        position.append(row[2])
        squad.append(row[3])
        competition.append(row[4])
        age.append(int(row[5]))
        born.append(float(row[6]))
        time.append(float(row[7]))
        goal.append(float(row[8]))
        assist.append(float(row[9]))
        dribbling.append(float(row[10]))
        tackle.append(float(row[11]))

# Crear un grafo no dirigido
G = nx.Graph()

def calculate_edge_weight(i, j):
    return abs(goal[i] - goal[j]) + abs(assist[i] - assist[j]) + abs(dribbling[i] - dribbling[j]) + abs(tackle[i] - tackle[j])

def show_table(sorted_players, input_player):
    # Conjunto para mantener un registro de jugadores ya insertados
    inserted_players = set()

    # Limpiar el contenido anterior de la tabla (si lo hay)
    for item in tree.get_children():
        tree.delete(item)

    # Insertar datos en la tabla
    for u, v, weight in sorted_players:
        # Redondear el peso a 4 decimales
        rounded_weight = round(weight, 4)
        # Verificar si el jugador ya fue insertado
        if u == input_player:
            index = player.index(v)
            data = [v, nation[index], position[index], squad[index], competition[index], age[index], rounded_weight]
            tree.insert("", "end", values=data)
            # Agregar el jugador al conjunto de jugadores insertados
            inserted_players.add(v)
        else:
            # Si v está repetido, utilizar u en lugar de v
            index = player.index(u)
            data = [u, nation[index], position[index], squad[index], competition[index], age[index], rounded_weight]
            tree.insert("", "end", values=data)
            # Agregar el jugador al conjunto de jugadores insertados
            inserted_players.add(u)

def run_prim_algorithm():
    # Obtener el nombre del jugador ingresado
    input_player = player_entry.get()

    # Verificar si el jugador está en la lista
    if input_player not in player:
        result_label.config(text="El jugador no está en la lista.")
        return

    # Obtener la posición del jugador ingresado
    input_position = position[player.index(input_player)]

    # Obtener la liga seleccionada en ese momento
    selected_league = league_var.get()

    # Obtener el intervalo de edades
    age_interval = age_entry.get()
    
    if not age_interval:
        # Si el input de edades está vacío, considerar el intervalo como neutral
        min_age, max_age = float('-inf'), float('inf')
    else:
        try:
            min_age, max_age = map(int, age_interval.split('-'))
        except ValueError:
            result_label.config(text="Formato de intervalo de edades incorrecto.")
            return

    # Limpiar el grafo antes de volver a construirlo
    G.clear()

    # Añadir nodos y aristas al grafo
    for i in range(len(player)):
        if (
            position[i] == input_position
            and (selected_league == "" or competition[i] == selected_league)
            and min_age <= age[i] <= max_age
        ):
            G.add_node(player[i])
            weight = calculate_edge_weight(player.index(input_player), i)
            G.add_edge(input_player, player[i], weight=weight)

    # Nodos y aristas del árbol de expansión mínima
    T = set()
    # Nodos visitados
    U = {input_player}

    # Obtener todas las aristas relevantes desde el principio
    edges = [(u, v, G[u][v]['weight']) for u in G.nodes() for v in G.neighbors(u) if v not in U and position[player.index(v)] == input_position]

    # Mientras U no sea igual a todos los nodos
    while U != set(player):
        # Verificar si hay aristas disponibles
        if not edges:
            break

        # Encontrar la arista de menor costo (u, v)
        min_edge = min(edges, key=lambda x: x[2])
        u, v, weight = min_edge

        # Agregar la arista al árbol
        T.add((u, v, weight))
        # Agregar el nuevo nodo v al conjunto U
        U.add(v)

        # Eliminar la arista seleccionada de la lista de aristas
        edges.remove(min_edge)

    # Obtener los 10 jugadores con menor peso
    sorted_players = sorted(T, key=lambda x: x[2])[:10]

    # Mostrar la tabla con los 10 jugadores con menor peso
    show_table(sorted_players, input_player)

    print(sorted_players)

# Configuración de la interfaz gráfica
root = tk.Tk()
root.title("Algoritmo de Prim")
root.state('zoomed')  # Maximizar la ventana

# Etiqueta y entrada para el nombre del jugador
player_label = tk.Label(root, text="Ingrese el nombre del jugador:")
player_label.pack(pady=10)
player_entry = tk.Entry(root)
player_entry.pack(pady=10)

# Lista de ligas disponibles
leagues = ["", "Ligue 1", "Bundesliga", "Serie A", "La Liga", "Premier League"]

# Variable para almacenar la liga seleccionada
league_var = tk.StringVar()

# Etiqueta y menú desplegable para seleccionar la liga
league_label = tk.Label(root, text="Seleccionar Liga:")
league_label.pack(pady=5)
league_menu = ttk.Combobox(root, textvariable=league_var, values=leagues)
league_menu.pack(pady=10)
league_menu.set("")  # Configurar la liga por defecto como vacía

# Etiqueta y entrada para el intervalo de edades
age_label = tk.Label(root, text="Introducir intervalo de edades (ej. 20-30):")
age_label.pack(pady=5)
age_entry = tk.Entry(root)
age_entry.pack(pady=10)

# Botón para ejecutar el algoritmo de Prim
run_button = tk.Button(root, text="Ejecutar Prim", command=run_prim_algorithm)
run_button.pack(pady=10)

# Etiqueta para mostrar el resultado del árbol de expansión mínima
result_label = tk.Label(root, text="")
result_label.pack(pady=10)

# Crear el árbol (tabla)
headers = ["Jugador", "Nación", "Posición", "Equipo", "Competición", "Edad", "Similitud"]
tree = ttk.Treeview(root, columns=headers, show="headings")

# Configurar encabezados de columna
for header in headers:
    tree.heading(header, text=header)
    tree.column(header, anchor=tk.CENTER)

# Configurar el tamaño de las filas
tree['height'] = 10  # Puedes ajustar este valor según sea necesario

# Configurar el tamaño de las columnas
for header in headers:
    tree.column(header, width=150)  # Puedes ajustar el ancho según sea necesario

# Colocar la tabla en la ventana principal
tree.pack()

# Iniciar el bucle principal
root.mainloop()

[('Ben Chilwell', 'Vitaliy Mykolenko', 0.4199999999999999), ('Ben Chilwell', 'Reece James', 1.27), ('Ben Chilwell', 'Sergio Gómez', 1.73), ('Ben Chilwell', 'Nathan Patterson', 2.12), ('Ben Chilwell', 'Mads Bech Sørensen', 2.9299999999999997), ('Ben Chilwell', 'Rúben Vinagre', 4.6)]
